In [223]:
#libraries to import
import requests
import os
import pandas as pd
import json
import time
import numpy

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [224]:
#create a function to get the 50 closest POIs near a specific bike station from Foursquare

def fs_get_station(latitude,longitude):
    url = "https://api.foursquare.com/v3/places/search"
    params = {"ll":f"{latitude},{longitude}",
    "radius":"1000",
    "sort":"DISTANCE"}
    headers = {
    "Accept": "application/json",
    "Authorization": os.getenv('foursquare_key')}
    return requests.request("GET", url, params=params, headers=headers)

#load list of stations
stations = pd.read_json('stations.json')

#retrieve all info for each station and add it to a json
fs_response=0
fs_data=[]
for i in range(len(stations)):
    #get a response for each station:
    fs_response=fs_get_station(stations['latitude'][i],stations['longitude'][i])
    
    #add the latest response to the previous ones  
    fs_data+=fs_response.json()['results']
    
    #wait 3 seconds before sending the next request
    print('API calls running, please wait')
    time.sleep(3)
print('API calls completed')
   

API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls completed


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [225]:
#show all the columns contained in the Dataframes
pd.set_option('display.max_columns', None)

#create DataFrame with all POIs returned by Foursquare
fs_pois = pd.json_normalize(fs_data)
fs_pois.head()

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.census_block,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.postcode,location.region,location.address_extended,related_places.parent.fsq_id,related_places.parent.categories,related_places.parent.name,related_places.children,geocodes.front_door.latitude,geocodes.front_door.longitude,location.po_box
0,53cd978c498e97070652552b,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,26,/v3/places/53cd978c498e97070652552b,GWARbar,America/New_York,37.548903,-77.442898,37.548700,-77.443018,37.548700,-77.443018,217 W Clay St,517600302003023,US,Brook Rd,Richmond-Petersburg,"217 W Clay St (Brook Rd), Richmond, VA 23220",Richmond,23220,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c5da1657f661b8d14de4b1c,"[{'id': 12101, 'name': 'Church', 'short_name':...",[],VeryLikelyOpen,189,/v3/places/4c5da1657f661b8d14de4b1c,Ebenezer Baptist Church,America/New_York,37.550260,-77.442148,37.550469,-77.442025,37.550469,-77.442025,216 W Leigh St,517600302003001,US,NaN,Richmond-Petersburg,"216 W Leigh St, Richmond, VA 23220",Richmond,23220,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53625ed4498ebfdb64fc9cc1,"[{'id': 13034, 'name': 'Café', 'short_name': '...",[],Unsure,229,/v3/places/53625ed4498ebfdb64fc9cc1,Saison Market,America/New_York,37.546885,-77.442172,37.546736,-77.442023,37.546736,-77.442023,323 N Adams St,517600302004009,US,,Richmond-Petersburg,"323 N Adams St, Richmond, VA 23220",Richmond,23220,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4ad874acf964a520821121e3,"[{'id': 13034, 'name': 'Café', 'short_name': '...",[],LikelyOpen,239,/v3/places/4ad874acf964a520821121e3,Lift Coffee Shop & Cafe,America/New_York,37.547086,-77.444214,37.547235,-77.444127,37.547235,-77.444127,218 W Broad St,517600302003027,US,btwn N Jefferson & Madison St,Richmond-Petersburg,218 W Broad St (btwn N Jefferson & Madison St)...,Richmond,23220,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c9eb36b7c096dcb7d16d8d1,"[{'id': 12000, 'name': 'Community and Governme...",[],LikelyOpen,244,/v3/places/4c9eb36b7c096dcb7d16d8d1,Richmond VA Fire Station 5,America/New_York,NaN,NaN,37.551040,-77.442907,37.551040,-77.442907,324 W Leigh St,517600302003002,US,NaN,Richmond-Petersburg,"324 W Leigh St, Richmond, VA 23220",Richmond,23220,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
#column 'categories' is not formatted properly and needs further manipulation

#convert column 'categories' to series, keeping only the first category name
categ=pd.json_normalize(pd.json_normalize(fs_pois['categories'])[0])['name']
print(categ)

0                           Bar
1                        Church
2                          Café
3                          Café
4      Community and Government
                 ...           
222         American Restaurant
223                 Coffee Shop
224                 Coffee Shop
225     New American Restaurant
226          Italian Restaurant
Name: name, Length: 227, dtype: object


Put your parsed results into a DataFrame

In [227]:
#select the relevant column names from the POIs DataFrame
fs_cols=['distance','name','geocodes.main.latitude','geocodes.main.longitude','location.address']

#put the relevant columns from the POIs DataFrame and the categories series in a new Dataframe which will be used use for modelling
FS_POIS = fs_pois[fs_cols].copy()
FS_POIS['categories']=categ

#save POIs Dataframe to json file
FS_POIS.to_json('FS_POIS.json')

print(FS_POIS.shape)
FS_POIS.head()

(227, 6)


,distance,name,geocodes.main.latitude,geocodes.main.longitude,location.address,categories
0,26,GWARbar,37.548700,-77.443018,217 W Clay St,Bar
1,189,Ebenezer Baptist Church,37.550469,-77.442025,216 W Leigh St,Church
2,229,Saison Market,37.546736,-77.442023,323 N Adams St,Café
3,239,Lift Coffee Shop & Cafe,37.547235,-77.444127,218 W Broad St,Café
4,244,Richmond VA Fire Station 5,37.551040,-77.442907,324 W Leigh St,Community and Government


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [228]:
#create a function to get the 50 closest POIs near a specific bike station from Yelp

def yelp_get_station(latitude,longitude):
    url = f'https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=1000&sort_by=distance&limit=50'
    headers = {
    "Accept": "application/json",
    "Authorization": f'bearer {os.getenv("yelp_key")}'
    }
    return requests.request("GET", url, headers=headers)

#load list of stations
stations = pd.read_json('stations.json')

#retrieve all info for each station and add it to a json
yelp_response=0
yelp_data=[]
for i in range(len(stations)):
    #get a response for each station:
    yelp_response=yelp_get_station(stations['latitude'][i],stations['longitude'][i])
    
    #add the latest response to the previous ones  
    yelp_data+=yelp_response.json()['businesses']
    
    #wait 3 seconds before sending the next request
    print('API calls running, please wait')
    time.sleep(3)
print('API calls completed')
   

API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls running, please wait
API calls completed


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [268]:
#show all the columns contained in the Dataframes
pd.set_option('display.max_columns', None)

#create DataFrame with all POIs returned by Yelp
yelp_pois = pd.json_normalize(yelp_data)
yelp_pois.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price
0,NV5Nh1bN7AEGWOztjXCZnA,abner-clay-park-richmond,Abner Clay Park,https://s3-media4.fl.yelpcdn.com/bphoto/iQYqaR...,False,https://www.yelp.com/biz/abner-clay-park-richm...,1,"[{'alias': 'tennis', 'title': 'Tennis'}, {'ali...",3.0,[],,,18.736233,37.549500,-77.442210,200 W Clay St,,,Richmond,23220,US,VA,"[200 W Clay St, Richmond, VA 23220]",NaN
1,68nlZQoRap5HZIxY7Kj3qA,gwarbar-richmond-3,GWARbar,https://s3-media2.fl.yelpcdn.com/bphoto/JuBmb5...,False,https://www.yelp.com/biz/gwarbar-richmond-3?ad...,220,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"[delivery, pickup]",+18049189352,(804) 918-9352,32.409206,37.548760,-77.443033,217 W Clay St,,,Richmond,23220,US,VA,"[217 W Clay St, Richmond, VA 23220]",$$
2,7X4pz9A_uKCfXmox76v5vg,pennys-wine-shop-richmond,Penny's Wine Shop,https://s3-media2.fl.yelpcdn.com/bphoto/iIVC7P...,False,https://www.yelp.com/biz/pennys-wine-shop-rich...,18,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",5.0,[],,,113.870430,37.547860,-77.442548,405 Brook Rd,,None,Richmond,23220,US,VA,"[405 Brook Rd, Richmond, VA 23220]",NaN
3,I1Or5SrdTRrHuBpGU6ydvA,black-history-museum-and-cultural-center-of-vi...,Black History Museum and Cultural Center of Vi...,https://s3-media4.fl.yelpcdn.com/bphoto/i7bK2g...,False,https://www.yelp.com/biz/black-history-museum-...,20,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,[],+18047809093,(804) 780-9093,174.744711,37.550125,-77.441315,122 W Leigh St,,,Richmond,23220,US,VA,"[122 W Leigh St, Richmond, VA 23220]",NaN
4,8iwT9kd_Fe9emCuXmbhNMw,lift-coffee-shop-and-cafe-richmond,Lift Coffee Shop & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/aydwES...,False,https://www.yelp.com/biz/lift-coffee-shop-and-...,224,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,[delivery],+18043445438,(804) 344-5438,212.868380,37.547180,-77.444140,218 W Broad St,,,Richmond,23220,US,VA,"[218 W Broad St, Richmond, VA 23220]",$$


In [230]:
#just as with the response from Foursquare, the 'categories' column is not formatted properly and needs further manupulation

#convert column 'categories' to series, keeping only the category title
catego=pd.json_normalize(pd.json_normalize(yelp_pois['categories'])[0])['title']
print(catego)

0              Tennis
1            Barbeque
2           Wine Bars
3             Museums
4        Coffee & Tea
            ...      
946           Italian
947          Japanese
948             Pizza
949             Parks
950    American (New)
Name: title, Length: 951, dtype: object


Put your parsed results into a DataFrame

In [269]:
#select the relevant column names from the POIs DataFrame
yelp_cols=['name','rating','review_count','distance','coordinates.latitude','coordinates.longitude','location.address1','price']

#put the relevant columns from the POIs DataFrame and the categories series in a new Dataframe which will be used use for modelling
YELP_POIS = yelp_pois[yelp_cols].copy()
YELP_POIS['categories']=catego

#save POIs Dataframe to json file
YELP_POIS.to_json('YELP_POIS.json')

print(YELP_POIS.shape)
YELP_POIS.head()

(951, 9)


,name,rating,review_count,distance,coordinates.latitude,coordinates.longitude,location.address1,price,categories
0,Abner Clay Park,3.0,1,18.736233,37.549500,-77.442210,200 W Clay St,NaN,Tennis
1,GWARbar,4.0,220,32.409206,37.548760,-77.443033,217 W Clay St,$$,Barbeque
2,Penny's Wine Shop,5.0,18,113.870430,37.547860,-77.442548,405 Brook Rd,NaN,Wine Bars
3,Black History Museum and Cultural Center of Vi...,4.5,20,174.744711,37.550125,-77.441315,122 W Leigh St,NaN,Museums
4,Lift Coffee Shop & Cafe,4.0,224,212.868380,37.547180,-77.444140,218 W Broad St,$$,Coffee & Tea


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp and Foursquare return almost identical information, although the coordinates of the distances from the bike stations to the venues/businesses differ slightly. Additionally, Yelp returns around 4 times more results than Foursquare, and also returns the user rating and 'price' level of each venue/business. All things considered, Yelp is a much better option for statistical modelling than Foursquare.

Get the top 10 restaurants according to their rating

In [266]:
"""Yelp returns the ratings but its categories cannot be used to filter restaurants directly. 
Then, each Yelp category needs to be sorted into Foursquare categories.
Foursquare category 'Restaurants' can be used to find the top 10 restaurants according to their rating
"""

#create sets of categories for each API
categories_places_FS=list(FS_POIS['categories'].unique())
categories_businesses_YELP=list(YELP_POIS['categories'].unique())

#using AI assistance(OpenAI), sort all Business category names in categories_places_FS into Places category names in categories_places_FS
"""
AI returned:

Business: Place,
    Tennis: Sports & Recreation,
    Barbeque: Restaurant,
    Wine Bars: Wine Bar,
    Museums: History Museum,
    Coffee & Tea: Coffee Shop,
    list continues...
    
To maintain readability the complete list was saved as 'GPT_category_mapping.csv'
"""

#create a DataFrame "GPT_df", containing the business and places categories
GPT_category_mapping = pd.read_csv('GPT_category_mapping.csv',escapechar='\\')
GPT_df=pd.DataFrame(columns=['Business_Cate', 'Place_Cate'], index=range(len(GPT_category_mapping)))
for i in range(len(GPT_category_mapping)):
    GPT_df['Business_Cate'][i]=GPT_category_mapping['Business: Place'].iloc[i].lstrip().split(': ')[0]
    GPT_df['Place_Cate'][i]=GPT_category_mapping['Business: Place'].iloc[i].lstrip().split(': ')[1]
GPT_df.to_json('GPT_df.json')
    
#create a list of all the Yelp business categories corresponding to Foursquare "Restaurant" category
Yelp_Rests=GPT_df[GPT_df['Place_Cate']=='Restaurant']

#find the top 10 rated businesses with 'Place' category =  "Restaurants", sorte by descending rating and ascending distance to the nearest bike station
Top_10=YELP_POIS[YELP_POIS['categories'].isin(Yelp_Rests['Business_Cate'])].sort_values(['rating','distance'], ascending=[False,True]).head(10).reset_index()
Top_10

,index,name,rating,distance,coordinates.latitude,coordinates.longitude,location.address1,price,categories
0,417,Red Oak Cafe,5.0,60.985735,37.540450,-77.433270,Broad Stte Rd & Hckt,NaN,Restaurants
1,342,Copper Kettle,5.0,115.661916,54.388401,-1.941740,Galway House,NaN,Restaurants
2,905,Domoishi,5.0,123.625991,37.546072,-77.455833,1106 W Main St,NaN,Asian Fusion
3,736,The Smoky Mug,5.0,127.120958,37.571247,-77.432047,15 E Brookland Park Blvd,NaN,Barbeque
4,357,Kat’s Breakfast & Lunch,5.0,150.689525,37.540102,-77.437711,701 E Franklin St,NaN,Breakfast & Brunch
5,5,Blue Bottle Restaurant,5.0,232.373600,37.546921,-77.443680,202 W Broad St,NaN,Restaurants
6,443,Copper Kettle,5.0,246.506367,54.388401,-1.941740,Galway House,NaN,Restaurants
7,286,Westham Porch,5.0,303.879711,37.535730,-77.435050,100 South 12th Street,NaN,American (New)
8,146,Govinda Restaurant,5.0,349.934120,37.550910,-77.448210,812 W Marshall St,NaN,Vegan
9,621,Sear Burger,5.0,355.441779,37.545137,-77.439318,321 N 2nd St,NaN,Burgers
